# 03 — Mocking & Patching

This notebook covers:
- `unittest.mock.patch` and `MagicMock`
- `monkeypatch` for env vars and attributes
- `responses` library for HTTP mocking
- When to mock vs. when to use real objects

## Setup

In [ ]:
import subprocess, sys, textwrap, tempfile, pathlib

def run_pytest(test_code: str, extra_args: list[str] | None = None, extra_files: dict[str, str] | None = None):
    with tempfile.TemporaryDirectory() as td:
        td_path = pathlib.Path(td)
        p = td_path / "test_tmp.py"
        p.write_text(textwrap.dedent(test_code))
        if extra_files:
            for name, content in extra_files.items():
                (td_path / name).write_text(textwrap.dedent(content))
        cmd = [sys.executable, "-m", "pytest", str(td), "-v", "--tb=short", "--no-header"]
        if extra_args:
            cmd.extend(extra_args)
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=30)
        print(result.stdout + result.stderr)
        return result.returncode

print("Helper ready.")

## 1. MagicMock Basics

`MagicMock` creates objects that record how they're used.

In [ ]:
from unittest.mock import MagicMock

# Create a mock
model = MagicMock()

# Call it — doesn't crash, returns another MagicMock
result = model.predict(["hello"])
print(f"result type: {type(result)}")

# Configure return value
model.predict.return_value = [1, 0, 1]
assert model.predict(["any", "input"]) == [1, 0, 1]

# Check it was called
model.predict.assert_called()
model.predict.assert_called_with(["any", "input"])
print(f"call count: {model.predict.call_count}")

print("\nMagicMock: records calls, configurable return values, never crashes.")

## 2. `patch` — Replacing Objects During Tests

In [ ]:
# Module to test: a function that calls an external API
module_code = '''
import requests

def fetch_user(user_id: int) -> dict:
    resp = requests.get(f"https://api.example.com/users/{user_id}")
    resp.raise_for_status()
    return resp.json()
'''

test_code = '''
from unittest.mock import patch, MagicMock
from mymodule import fetch_user

@patch("mymodule.requests.get")
def test_fetch_user(mock_get):
    # Configure the mock
    mock_response = MagicMock()
    mock_response.json.return_value = {"id": 1, "name": "Alice"}
    mock_response.raise_for_status.return_value = None
    mock_get.return_value = mock_response

    # Call the function — it uses mock instead of real requests
    result = fetch_user(1)

    assert result == {"id": 1, "name": "Alice"}
    mock_get.assert_called_once_with("https://api.example.com/users/1")
'''

rc = run_pytest(test_code, extra_files={"mymodule.py": module_code})
assert rc == 0
print("patch replaces requests.get with a mock — no real HTTP call!")

## 3. The Patch Target Rule

**Critical**: patch where the name is *looked up*, not where it's *defined*.

```python
# mymodule.py
from datetime import datetime  # <-- datetime is now in mymodule's namespace

# WRONG: @patch("datetime.datetime")
# RIGHT: @patch("mymodule.datetime")
```

In [ ]:
module_code = '''
from datetime import datetime

def get_greeting():
    hour = datetime.now().hour
    if hour < 12:
        return "Good morning"
    return "Good afternoon"
'''

test_code = '''
from unittest.mock import patch
from datetime import datetime
from mymod import get_greeting

@patch("mymod.datetime")
def test_morning(mock_dt):
    mock_dt.now.return_value = datetime(2024, 1, 1, 8, 0)
    assert get_greeting() == "Good morning"

@patch("mymod.datetime")
def test_afternoon(mock_dt):
    mock_dt.now.return_value = datetime(2024, 1, 1, 15, 0)
    assert get_greeting() == "Good afternoon"
'''

rc = run_pytest(test_code, extra_files={"mymod.py": module_code})
assert rc == 0
print("Patched mymod.datetime (where it's looked up), not datetime.datetime!")

## 4. `monkeypatch` for Environment & Attributes

In [ ]:
rc = run_pytest('''
import os

def get_model_path():
    return os.environ.get("MODEL_PATH", "/default/model.bin")

def test_default_path():
    assert get_model_path() == "/default/model.bin"

def test_custom_path(monkeypatch):
    monkeypatch.setenv("MODEL_PATH", "/custom/model.bin")
    assert get_model_path() == "/custom/model.bin"

def test_unset(monkeypatch):
    monkeypatch.delenv("MODEL_PATH", raising=False)
    assert get_model_path() == "/default/model.bin"

# monkeypatch for attributes
class Config:
    DEBUG = False

def test_debug_mode(monkeypatch):
    monkeypatch.setattr(Config, "DEBUG", True)
    assert Config.DEBUG is True
''')
assert rc == 0
print("monkeypatch: env vars, attributes, dict items — auto-reverted after test.")

## 5. `responses` — HTTP Mocking Made Easy

In [ ]:
rc = run_pytest('''
import responses
import requests

@responses.activate
def test_api_call():
    responses.add(
        responses.GET,
        "https://api.example.com/data",
        json={"items": [1, 2, 3]},
        status=200,
    )

    resp = requests.get("https://api.example.com/data")
    assert resp.status_code == 200
    assert resp.json()["items"] == [1, 2, 3]

@responses.activate
def test_api_error():
    responses.add(
        responses.GET,
        "https://api.example.com/data",
        json={"error": "not found"},
        status=404,
    )

    resp = requests.get("https://api.example.com/data")
    assert resp.status_code == 404

@responses.activate
def test_multiple_calls():
    # First call returns 500, second returns 200
    responses.add(responses.GET, "https://api.example.com/data", status=500)
    responses.add(responses.GET, "https://api.example.com/data", json={"ok": True}, status=200)

    r1 = requests.get("https://api.example.com/data")
    r2 = requests.get("https://api.example.com/data")
    assert r1.status_code == 500
    assert r2.status_code == 200
''')
assert rc == 0
print("responses: clean HTTP mocking with @responses.activate decorator.")

## 6. When to Mock vs. When to Use Real Objects

| Mock | Don't Mock |
|------|------------|
| HTTP APIs, LLM calls | Pure functions |
| Database in unit tests | Simple data classes |
| File system (sometimes) | Your own utilities |
| Time, randomness | Anything fast & deterministic |

**Rule of thumb**: mock at boundaries (network, disk, external services). Test your own code with real objects.

## Key Takeaways

1. **MagicMock**: records calls, configurable returns, never crashes
2. **`@patch("where.its.looked.up")`** — not where it's defined
3. **`monkeypatch`** for simple value swaps (env vars, attributes)
4. **`mock.patch`** when you need to verify interactions (assert_called)
5. **`responses`** for clean HTTP mocking
6. Mock at **boundaries**, test internals with real objects